In [1]:
import os, logging
from datetime import datetime

import mlflow
import numpy as np
import tensorflow as tf
from cloudpickle import Pickler
from mlflow.pyfunc import log_model
from csbdeep.models import Config, CARE

from flame.model import MLFLOW_CARE_Model

2025-06-10 02:16:12.404044: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-10 02:16:17.998531: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749546979.694117   20531 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749546980.351611   20531 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749546985.902384   20531 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
logger = logging.getLogger("main")
logging.basicConfig(
    filename=f"{datetime.now().strftime('%Y%m%d-%H%M%S')}_logger.log",
    encoding="utf-8",
    level=logging.DEBUG
)

In [10]:
TEST_MODEL_DIR = os.path.join(os.getcwd(), "test_model")
OTHER_MODEL_DIR = "/mnt/d/models/FLAME_CARE_5F-40F/0"

In [4]:
test_config = Config(
    axes='YXC',
)

In [11]:
model = CARE(
    None,
    OTHER_MODEL_DIR
)

Loading network weights from 'weights_best.h5'.


In [6]:
pyfunc_model = MLFLOW_CARE_Model(
    test_config,
    'test',
    basedir=TEST_MODEL_DIR
)

/home/balulab/miniconda3/envs/care/lib/python3.12/site-packages/csbdeep/models/base_model.py:203: UserWarning: output path for model already exists, files may be overwritten: /mnt/d/code/Balu_CARE/test_model/test
  warnings.warn('output path for model already exists, files may be overwritten: %s' % str(self.logdir.resolve()))


### Try pickling before training

In [7]:
Pickler(
    open(os.path.join(TEST_MODEL_DIR, "pickle.obj"), 'wb+')
).dump(model)

In [8]:
Pickler(
    open(os.path.join(TEST_MODEL_DIR, "pyfunc_pickle.obj"), 'wb+')
).dump(pyfunc_model)

PicklingError: __reduce__ must return a string or tuple

### Try Pickling after training

In [ ]:
X = np.random.uniform(size=(1,128,128,1))
Y = np.random.uniform(size=(1,128,128,1))
X_val, Y_val = X.copy(), Y.copy()

In [ ]:
pyfunc_model.train(X=X, Y=Y, validation_data = (X_val, Y_val), epochs=1)

I0000 00:00:1749545791.088241   16630 service.cc:152] XLA service 0x7f14d4006f30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1749545791.088284   16630 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-06-10 01:56:31.121932: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1749545791.330242   16630 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1749545794.404784   16630 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


399/400 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.2186 - mae: 0.2186 - mse: 0.0792

/home/balulab/miniconda3/envs/care/lib/python3.12/site-packages/keras/src/models/functional.py:238: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: input
Received: inputs=('Tensor(shape=(1, 128, 128, 1))',)
  warnings.warn(msg)


400/400 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - loss: 0.2183 - mae: 0.2183 - mse: 0.0790 - val_loss: 0.0508 - val_mae: 0.0508 - val_mse: 0.0092 - learning_rate: 4.0000e-04

Loading network weights from 'weights_best.h5'.


In [ ]:
Pickler(
    open(os.path.join(TEST_MODEL_DIR, "pyfunc_pickle.obj"), 'wb+')
).dump(pyfunc_model)

InvalidArgumentError: Cannot convert a Tensor of dtype resource to a NumPy array.